# TM10007 Assignment ADNI Dataset
Bram Gerritse 4462599  
Saskia Bijl 4439457  
Karan Ramsodit  
Enzo Kerkhof 44488555

In [20]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/Enzo-Kerkhof/TM10007_PROJECT.git

In [0]:
# Import used libraries
from pathlib import Path
import pandas as pd
import numpy as np

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [23]:
# Data loading functions. Uncomment the one you want to use
from adni.load_data import load_data

data = load_data()
print('The dataset ABNI contains the following\n')
print(f'The number of samples/patients: {len(data.index)}')
print(f'The number of columns/features: {len(data.columns)}')
print('\nHeader of the labels of the sampels\n')
print(data['label'].head(), type(data))
print('\nCurrent working dir:', Path.cwd())

The dataset ABNI contains the following

The number of samples/patients: 855
The number of columns/features: 268

Header of the labels of the sampels

ID
0_002_S_0413_bl_0    AD
0_002_S_0559_bl_0    AD
0_002_S_0619_bl_0    CN
0_002_S_0685_bl_0    AD
0_002_S_0816_bl_0    CN
Name: label, dtype: object <class 'pandas.core.frame.DataFrame'>

Current working dir: /content
